In [ ]:
!pip install tensorflow

In [ ]:
!pip install keras


In [ ]:
from keras.layers import Input,Lambda,Dense,Flatten
from keras.models import Model
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
image_size=[224,224]

In [ ]:
train_path='/content/drive/MyDrive/train'
test_path='/content/drive/MyDrive/test'

In [ ]:
#adding the pre processing layer infront of resnet
resnet=ResNet50(input_shape=image_size+[3],weights='imagenet',include_top=False)

94765736/94765736 [==============================] - 3s 0us/step


In [ ]:
#this is for not training the existing weights
for layer in resnet.layers:
  layer.trainable=False

In [ ]:
#this is useful for getting the number of classes
folders=glob('/content/drive/MyDrive/train/*')

In [ ]:
print(len(folders))

3


In [ ]:
#our layers
x=Flatten()(resnet.output)

In [ ]:
#x=Dense(1000,activation='relu')(x)
prediction=Dense(3,activation='softmax')(x)

In [ ]:
#creating the model object
model=Model(inputs=resnet.input,outputs=prediction)

In [ ]:
#to view the structure of our model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [ ]:
#we tell the model what cost and optimization to use
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

In [ ]:
test_datagen=ImageDataGenerator(rescale=1./255)

In [ ]:
training_set=train_datagen.flow_from_directory('/content/drive/MyDrive/train',target_size=(224,224),batch_size=32,class_mode='categorical')

Found 1120 images belonging to 2 classes.


In [ ]:
test_set=test_datagen.flow_from_directory('/content/drive/MyDrive/test',target_size=(224,224),batch_size=32,class_mode='categorical')

Found 320 images belonging to 2 classes.


In [ ]:
from keras.applications.resnet50 import preprocess_input
training_set = [(preprocess_input(image), label) for image, label in training_set]
test_set = [(preprocess_input(image), label) for image, label in test_set]

In [ ]:
print(len(training_set))
print(len(test_set))

In [ ]:
#fit the model
r=model.fit(training_set,validation_data=test_set,epochs=5,steps_per_epoch=len(training_set),validation_steps=None)